In [1]:
import pymysql as db
import numpy as np
import pandas as pd
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from sklearn.preprocessing import MinMaxScaler
import threading as t
import time
from sqlalchemy import create_engine

db.install_as_MySQLdb()

import MySQLdb

In [2]:
#출고 데이터 저장 딕셔너리
df = 0
#현재 날짜 가지고 오기
now = datetime.datetime.now()
#현재 날짜를 년,월,요일 등으로 분리
now_result = now.timetuple()
#요일
weekday = now_result.tm_wday
#월
month = now_result.tm_mon
#문구류_List
Stationery_list = []
#학습결과
result_dict = {"Stationery_NO" : [], "Count" : [], "Today" : []}
result_df = 0

In [3]:
class DB_Connect:
    conn = db.connect(host='localhost', user='root', password='1234', db='ai_db', charset='utf8')
    curs = conn.cursor(db.cursors.DictCursor)

In [4]:
#DB_SQL
class DB_SQL(DB_Connect):
    #DB값 읽기
    def db_read(self,sql):
        print(sql)
        conn = db.connect(host='localhost', user='root', password='1234', db='ai_db', charset='utf8')
        curs = conn.cursor(db.cursors.DictCursor)
        curs.execute(sql)
        print("db_read 성공!")
        result = curs.fetchall()
        df = pd.DataFrame(result)
        #문구류의 값 리스트에 넣어서 중복 제거
        Stationery_list = set(df['Stationery_NO'])
        curs.close()
        conn.close()
        
        return df, Stationery_list
    
    #DB값 쓰기
    def db_write(self,data):
        engine = create_engine("mysql+mysqldb://root:" + "1234" + "@localhost:3306/ai_db", encoding='utf-8')
        conn = engine.connect()
        data
        data.to_sql(name = "training_results", con = engine, if_exists='append', index=False)
        print('테이블 데이터 업데이트 완료!')
    
    #학습값 제거
    def db_del(self,sql):
        print(sql)
        conn = db.connect(host='localhost', user='root', password='1234', db='ai_db', charset='utf8')
        curs = conn.cursor()
        curs.execute(sql)
        print("DB_Data 제거 완료!")
        curs.close()
        conn.close()

In [5]:
#DB_Data_Read
def data_read(weekday):
    db_result = DB_SQL()
    
    print(weekday)
    
    if(weekday == 4 or weekday == 6 or weekday == 5):
        print("금요일 또는 토요일, 일요일은 휴일입니다.")
        weekday = 0
    else:
        weekday = weekday + 1
        
    sql = "select Stationery_NO, Count, MONTH(Date) as 'months' from ai_db.releases where Weekday = " + str(weekday) + " and MONTH(Date) = " +  str(month) + ";"       

    global df
    global Stationery_list

    df,Stationery_list = db_result.db_read(sql)

In [6]:
#학습
class AI_Model():
    #데이터셋 변환 과정
    def create_dataset(data, look_back=1):
        x_arr, y_arr = [], []
        
        for i in range(len(data) - look_back):
            x_arr.append(data[i:(i + look_back),0])
            y_arr.append(data[i + look_back, 0])
        
        x_arr = np.array(x_arr)
        x_arr = np.reshape(x_arr, (x_arr.shape[0], x_arr.shape[1],1))
        
        return x_arr, np.array(y_arr)
    
    #학습
    def training(no):
        #데이터 전처리 (0 ~ 1 범위)
        scaler = MinMaxScaler(feature_range=(0,1))
        #지금 전체 카운트롤 계산 했지만 Thread 문구류 각각해서 동시에 돌릴 예정
        data_temp = (df['Stationery_NO'] == no)
        data_result = df[data_temp]
        print(data_result)
        
        temp = df[data_temp].values.reshape(-1,1)
        trade_count = scaler.fit_transform(temp)
        
        #훈련
        train = trade_count[0:int(len(trade_count) * 0.5)]
        #검증
        val = trade_count[int(len(trade_count) * 0.5) : int(len(trade_count) *0.75)]
        #시험
        test = trade_count[int(len(trade_count) * 0.75) : -1]
        
        #x,y 트레이닝 셋
        x_train, y_train = AI_Model.create_dataset(train, 1)
        #x,y 검증셋
        x_val, y_val = AI_Model.create_dataset(val,1)
        #x,y 시험 셋
        x_test, y_text = AI_Model.create_dataset(val,1)
        
        #학습 모델 구성
        batch_size = 22
        model = Sequential()
        
        model.add(LSTM(50, return_sequences=True, input_shape=(1,1)))
        model.add(LSTM(64,return_sequences=False))
        model.add(Dense(1, activation='linear'))
        
        #손실
        model.compile(loss='mse', optimizer='rmsprop')
        model.summary()
        model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=50)
        predictions = model.predict(x_test, batch_size)
        read_predictions = scaler.inverse_transform(predictions)
        result = read_predictions[-1]
        
        print(str(no) + "의 문구류의 값은 " + str(result) +" 입니다.")
        
        #학습결과_Dict
        global result_dict
        result_dict['Stationery_NO'].append(no)
        result_dict['Count'].append(int(result))
        result_now = str(now_result.tm_year) + "-" + str(now_result.tm_mon) + "-" + str(now_result.tm_mday)
        result_dict['Today'].append(result_now)

In [7]:
#쓰레드 생성, 실행 함수
def run_threads(list_temp):
    threads = []
    
    for i in range(len(list_temp)):
        thread = t.Thread(target=AI_Model.training,kwargs={"no" : list_temp[i]})
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

In [8]:
data_read(weekday)

0
select Stationery_NO, Count, MONTH(Date) as 'months' from ai_db.releases where Weekday = 1 and MONTH(Date) = 12;
db_read 성공!


In [9]:
#모델 객체 생성
ai_model = AI_Model()

list_temp = list(Stationery_list)

start = time.time()

#학습 모델 쓰레드 생성
run_threads(list_temp)

#쓰레드 미생성
# for i in range(len(list_temp)):
#     ai_model.training(list_temp[i])

print('thread time : ', time.time() - start) 

      Count  Stationery_NO  months
0         2              2      12
13        1              2      12
20        1              2      12
21        1              2      12
26       53              2      12
36        6              2      12
37        3              2      12
41       45              2      12
45       12              2      12
52       29              2      12
55       17              2      12
69       16              2      12
85       34              2      12
90       53              2      12
93        1              2      12
107       5              2      12
117      11              2      12
119       1              2      12
122       1              2      12
127       1              2      12
128       3              2      12
131       3              2      12
138      55              2      12
163      12              2      12
165      24              2      12
169      15              2      12
174       2              2      12
180      44         

C:\Users\kkag2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\kkag2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\kkag2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\kkag2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\kkag2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to floa

Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________
Train on 299 samples, validate on 149 samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)     

336/336 [==============================] - 1s 2ms/sample - loss: 0.0117 - val_loss: 0.0207
Epoch 5/50
Epoch 7/50
297/297 [==============================] - 0s 2ms/sample - loss: 0.0193 - val_loss: 0.0167
Epoch 6/50
299/299 [==============================] - 0s 2ms/sample - loss: 0.0144 - val_loss: 0.0220
Epoch 6/50
311/311 [==============================] - 1s 2ms/sample - loss: 0.0182 - val_loss: 0.0210
Epoch 8/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0114 - val_loss: 0.0260
Epoch 6/50
297/297 [==============================] - 0s 2ms/sample - loss: 0.0192 - val_loss: 0.0167
Epoch 7/50
336/336 [==============================] - 1s 2ms/sample - loss: 0.0118 - val_loss: 0.0210
Epoch 8/50
299/299 [==============================] - 0s 2ms/sample - loss: 0.0143 - val_loss: 0.0222
Epoch 7/50
311/311 [==============================] - 1s 2ms/sample - loss: 0.0181 - val_loss: 0.0210
Epoch 9/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0103

311/311 [==============================] - 1s 2ms/sample - loss: 0.0181 - val_loss: 0.0210
Epoch 19/50
297/297 [==============================] - 0s 2ms/sample - loss: 0.0191 - val_loss: 0.0166
Epoch 19/50
299/299 [==============================] - 0s 2ms/sample - loss: 0.0142 - val_loss: 0.0219>.............] - ETA: 0s - loss: 0.006 - ETA: 0s - lo
Epoch 19/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0113 - val_loss: 0.0252
Epoch 17/50
311/311 [==============================] - 1s 2ms/sample - loss: 0.0182 - val_loss: 0.0210
Epoch 20/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0103 - val_loss: 0.0263
Epoch 17/50
336/336 [==============================] - 1s 2ms/sample - loss: 0.0116 - val_loss: 0.0207 - loss: 0.0098 - ETA: 0s - loss: 
Epoch 19/50
297/297 [==============================] - 0s 2ms/sample - loss: 0.0191 - val_loss: 0.0169
Epoch 20/50
299/299 [==============================] - 0s 2ms/sample - loss: 0.0142 - val_loss: 0.02

Epoch 31/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0104 - val_loss: 0.0269
Epoch 27/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0113 - val_loss: 0.0257
Epoch 27/50
336/336 [==============================] - 1s 2ms/sample - loss: 0.0116 - val_loss: 0.0209
Epoch 29/50
299/299 [==============================] - 0s 1ms/sample - loss: 0.0141 - val_loss: 0.0218
Epoch 31/50
297/297 [==============================] - 0s 2ms/sample - loss: 0.0192 - val_loss: 0.0166
Epoch 32/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0103 - val_loss: 0.0254
Epoch 28/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0113 - val_loss: 0.0263
Epoch 28/50
299/299 [==============================] - 0s 1ms/sample - loss: 0.0142 - val_loss: 0.0218
Epoch 32/5
336/336 [==============================] - 1s 2ms/sample - loss: 0.0116 - val_loss: 0.0207
Epoch 30/50
311/311 [==============================] - 0s 2ms/sample - los

311/311 [==============================] - 0s 2ms/sample - loss: 0.0181 - val_loss: 0.0212.............] - ETA: 0s - loss: 
Epoch 42/50
299/299 [==============================] - 0s 2ms/sample - loss: 0.0141 - val_loss: 0.0222
Epoch 43/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0114 - val_loss: 0.0256
Epoch 38/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0103 - val_loss: 0.0261
Epoch 38/50
297/297 [==============================] - 0s 2ms/sample - loss: 0.0191 - val_loss: 0.0166
Epoch 44/50
311/311 [==============================] - 0s 2ms/sample - loss: 0.0181 - val_loss: 0.0209
Epoch 43/50
299/299 [==============================] - 0s 2ms/sample - loss: 0.0143 - val_loss: 0.0221
Epoch 44/50
335/335 [==============================] - 1s 2ms/sample - loss: 0.0103 - val_loss: 0.0255
Epoch 39/50
286/336 [========================>.....] - ETA: 0s - loss: 0.0124335/335 [==============================] - 1s 2ms/sample - loss: 0.0113 - val_

In [10]:
result_df = pd.DataFrame(result_dict)
result_df.index +=1
result_df.index.names = ['id']
result_df

,Stationery_NO,Count,Today
id,,,
1,2,10,2019-12-2
2,1,8,2019-12-2
3,5,10,2019-12-2
4,6,10,2019-12-2
5,4,9,2019-12-2
6,3,10,2019-12-2


In [12]:
db_result = DB_SQL()

db_result.db_write(result_df)

테이블 데이터 업데이트 완료!
